In [16]:
import pandas as pd
import numpy as np
from psaw import PushshiftAPI
import datetime as dt
from bs4 import BeautifulSoup 
import requests
import re
import time
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import accuracy_score
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import confusion_matrix
from sklearn import svm, datasets
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

In [16]:
url = 'https://api.adzuna.com:443/v1/api/jobs/us/search/1?app_id=d3330ea8&app_key=cbcbe79274381fae5af07fb6828067ce%09&where=Houston%2C%20TX&max_days_old=365'

In [27]:
url_list=[]
for page in range(1,10):
    try: url_list.append("https://api.adzuna.com:443/v1/api/jobs/us/search/"+ str(page) +"?app_id=d3330ea8&app_key=cbcbe79274381fae5af07fb6828067ce&results_per_page=100&where=florida&max_days_old=100&sort_direction=up&sort_by=date")
    except:
        pass

In [29]:
spot = 0
initial_data = []
for url in url_list:
    initial_data.append(requests.get(url).json())
    print(str(spot) + ' missisipi')
    spot += 1
    time.sleep(1)

0 missisipi
1 missisipi
2 missisipi
3 missisipi
4 missisipi
5 missisipi
6 missisipi
7 missisipi
8 missisipi


In [66]:
dfMichaelFlorida = []
for x in range(0, len(initial_data)):
    for y in initial_data[x]['results']:
        dfMichaelFlorida.append(y)

In [110]:
newdf = []
for x in range(0, len(dfMichaelFlorida)):
    newdf.append(dfMichaelFlorida[x]['category'])
    

In [121]:
newestdf = []
for x in range(0, len(newdf)):
    newestdf.append( newdf[x]['label'])
newestdf = pd.DataFrame(newestdf, columns=['label'])

In [123]:
newestdf['label'].value_counts()

Unknown                             173
PR, Advertising & Marketing Jobs     84
Healthcare & Nursing Jobs            80
IT Jobs                              14
Engineering Jobs                      9
Consultancy Jobs                      7
Sales Jobs                            5
Retail Jobs                           3
Admin Jobs                            2
Customer Services Jobs                1
Teaching Jobs                         1
Trade & Construction Jobs             1
Manufacturing Jobs                    1
Name: label, dtype: int64

In [17]:
res = requests.get(url)
res.status_code

200

In [25]:
initial_data[0]

{'results': [{'created': '2018-10-02T02:14:35Z',
   'description': 'POSITION SUMMARY: HIM Coders are responsible for coding medical records according to ICD-10-CM and CPT coding guidelines. The primary functions of this position are: to review medical record documentation for inpatient encounters, assigning appropriate diagnostic and procedural codes and modifiers; and capture of quality management and statistical data elements for organizational reporting. Codes are assigned as appropriate to each diagnosis and procedure documented in the medical record by the…',
   'location': {'display_name': 'Florida',
    'area': ['US', 'Florida', 'Florida'],
    '__CLASS__': 'Adzuna::API::Response::Location'},
   'id': 969248072,
   'title': 'Clinical Coding Specialist III (Inpatient)',
   'category': {'tag': 'unknown',
    'label': 'Unknown',
    '__CLASS__': 'Adzuna::API::Response::Category'},
   'adref': 'eyJhbGciOiJIUzI1NiJ9.eyJzIjoiaFVNNktNUDZUbC05blJyMDJLVktSdyIsImkiOjk2OTI0ODA3Mn0.8LTPQltL

In [35]:
jsondict = res.json()
jsondict.keys()

dict_keys(['__CLASS__', 'results', 'mean', 'count'])

In [1]:
results=[]
for x in jsondict['results']:
    results.append(x)

NameError: name 'jsondict' is not defined

In [2]:
df = pd.DataFrame()

for x in range(0,len(results)):
    df[''(results[x]['title'])

SyntaxError: unexpected EOF while parsing (<ipython-input-2-94dbb87a5b42>, line 4)